<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-experiments/web_rag_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/deepset-ai/haystack.git@update_prompt_builder#egg=farm-haystack[preview]

DEPRECATION: git+https://github.com/deepset-ai/haystack.git@update_prompt_builder#egg=farm-haystack[preview] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import getpass
import os
import json
import logging

from haystack.preview.components.builders.prompt_builder import PromptBuilder
from haystack.preview.components.fetchers import LinkContentFetcher
from haystack.preview.components.generators.chat.openai import GPTChatGenerator
from haystack.preview.components.file_converters import HTMLToDocument
from haystack.preview.components.websearch import SerperDevWebSearch
from haystack.preview.components.rankers import TransformersSimilarityRanker
from haystack.preview.components.preprocessors import DocumentSplitter
from haystack.preview.dataclasses.chat_message import ChatMessage
from haystack.preview import Pipeline

In [3]:
serper_dev_api_key = getpass.getpass("Enter Serpdev api key:")
llm_api_key = getpass.getpass("Enter LLM provider api key:")

Enter Serpdev api key:··········
Enter LLM provider api key:··········


In [4]:


web_search = SerperDevWebSearch(api_key=serper_dev_api_key, top_k=5)
lcf = LinkContentFetcher(user_agents=["Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36"])
html_converter = HTMLToDocument()
text_splitter = DocumentSplitter(split_length=100, split_overlap=5)
similarity = TransformersSimilarityRanker(top_k=10)

template = """Given the information below: \n
            {% for document in documents %}
                {{ document.content }}
            {% endfor %}
            Answer question: {{ query }}. \n Answer:"""

prompt_builder = PromptBuilder(template_variables=["documents", "query"])
llm = GPTChatGenerator(api_key=llm_api_key, model_name="gpt-3.5-turbo")

In [5]:
pipe = Pipeline()
pipe.add_component("search", web_search)
pipe.add_component("fetcher", lcf)
pipe.add_component("converter", html_converter)
pipe.add_component("text_splitter", text_splitter)
pipe.add_component("similarity_ranker", similarity)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)


pipe.connect("search.links", "fetcher.urls")
pipe.connect("fetcher.streams", "converter.sources")
pipe.connect("converter.documents", "text_splitter.documents")
pipe.connect("text_splitter.documents", "similarity_ranker.documents")
pipe.connect("similarity_ranker.documents", "prompt_builder.documents")
pipe.connect("prompt_builder.prompt", "llm.messages")


In [6]:
logging.disable(logging.CRITICAL)

messages = [ChatMessage.from_system("You'll be asked questions in English but always respond in German. "
                                    "Remember to always translate your responses to German."),
            ChatMessage.from_user(template)]
query = "Why did Matt Gaetz seek to remove Kevin McCarthy as House Speaker?"
query_dict = {"query": query}

pipe.run(data={"prompt_builder": {"query": query, "messages": messages},
               "similarity_ranker": query_dict,
               "search": query_dict})

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


{'llm': {'replies': [ChatMessage(content='Matt Gaetz möchte Kevin McCarthy als Vorsitzenden des Repräsentantenhauses entfernen, weil er der Meinung ist, McCarthy habe sich nicht ausreichend für Ausgabenkürzungen eingesetzt. Gaetz behauptet auch, dass McCarthy eine Geheimvereinbarung mit Präsident Biden getroffen habe, um Hilfe für die Ukraine zu sichern, während er einen Maßnahmenplan unterstützte, der keine Hilfe für das kriegszerrüttete Land beinhaltet. Gaetz ist auch der Ansicht, dass McCarthy seine Versprechen gegenüber den Republikanern gebrochen hat, die er zu Beginn seiner Amtszeit als Sprecher abgegeben hat.', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, metadata={'model': 'gpt-3.5-turbo-0613', 'index': 0, 'finish_reason': 'stop', 'usage': {'prompt_tokens': 1424, 'completion_tokens': 154, 'total_tokens': 1578}})]}}